In [1]:
import pandas as pd
import tensorflow
import keras


In [2]:
from keras.utils import pad_sequences
from keras.models import Model,Sequential
from keras.layers import Dense,SimpleRNN,Embedding,LSTM,GRU,Bidirectional,TimeDistributed,InputLayer
from sklearn.model_selection import train_test_split
import nltk
from nltk.stem import WordNetLemmatizer,SnowballStemmer
from nltk.corpus import brown,treebank

In [3]:
 nltk.download('brown')

[nltk_data] Downloading package brown to
[nltk_data]     C:\Users\ravin\AppData\Roaming\nltk_data...
[nltk_data]   Package brown is already up-to-date!


True

In [4]:
brown

<CategorizedTaggedCorpusReader in 'C:\\Users\\ravin\\AppData\\Roaming\\nltk_data\\corpora\\brown'>

In [5]:
len(brown.sents())

57340

In [6]:
' '.join(brown.sents()[0])


"The Fulton County Grand Jury said Friday an investigation of Atlanta's recent primary election produced `` no evidence '' that any irregularities took place ."

In [7]:
 nltk.download('treebank')

[nltk_data] Downloading package treebank to
[nltk_data]     C:\Users\ravin\AppData\Roaming\nltk_data...
[nltk_data]   Package treebank is already up-to-date!


True

In [8]:
len(treebank.sents())

3914

In [9]:
 nltk.download('universal_tagset')

[nltk_data] Downloading package universal_tagset to
[nltk_data]     C:\Users\ravin\AppData\Roaming\nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!


True

In [10]:
d1 = brown.tagged_sents(tagset='universal')

In [11]:
d2 = treebank.tagged_sents(tagset='universal')

In [12]:
dataset = d1+d2

In [13]:
X = []
y = []
for l in dataset:
    x1=[]
    y1 =[]
    for t in l:
        x1.append(t[0])
        y1.append(t[1])
    X.append(x1)
    y.append(y1)
    

In [14]:
c=10
for i in range(len(X)):
    if len(X[i]) == len(y[i]):
        pass
    else:
        c+=1
        
c   
    

10

In [15]:
#finding vocabulary
voc = []
for doc in X:
    for word in doc:
        if word.lower() not in voc:
            voc.append(word.lower())

In [16]:
len(voc)

53232

In [17]:
voc = []
for doc in y:
    for word in doc:
        if word.lower() not in voc:
            voc.append(word.lower())

In [18]:
len(voc)

12

In [19]:

from tensorflow.keras.preprocessing.text import Tokenizer

In [20]:
tk_x = Tokenizer(oov_token='ovv')
tk_x.fit_on_texts(X)
tk_y = Tokenizer()
tk_y.fit_on_texts(y)
   

In [22]:
len(tk_x.word_index)

53233

In [23]:
len(tk_y.word_index)

12

In [24]:
tk_y.word_index

{'noun': 1,
 'verb': 2,
 '.': 3,
 'adp': 4,
 'det': 5,
 'adj': 6,
 'adv': 7,
 'pron': 8,
 'conj': 9,
 'prt': 10,
 'num': 11,
 'x': 12}

In [25]:
X_d = tk_x.texts_to_sequences(X)

In [26]:
 y_d = tk_y.texts_to_sequences(y)

In [27]:
len(X_d)

61254

In [28]:
len(y_d)

61254

In [29]:
final_x_d = pad_sequences(X_d,padding='post')
final_y_d = pad_sequences(y_d,padding='post')

In [30]:
final_x_d.shape

(61254, 271)

In [31]:
final_y_d.shape

(61254, 271)

In [32]:
from keras.utils import to_categorical

In [33]:
final_y = to_categorical(final_y_d)

In [34]:
final_y.shape

(61254, 271, 13)

In [35]:
final_x_d.shape

(61254, 271)

In [36]:
model = Sequential()
model.add(InputLayer(shape=(271,)))
model.add(Embedding(input_dim=53232+1,output_dim=5))
model.add(SimpleRNN(units=50,return_sequences=True))
model.add(TimeDistributed(Dense(units=13,activation='softmax')))

In [37]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ (None, 271, 5)              │         266,165 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ simple_rnn (SimpleRNN)               │ (None, 271, 50)             │           2,800 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed (TimeDistributed)   │ (None, 271, 13)             │             663 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 269,628 (1.03 MB)

 Trainable params: 269,628 (1.03 MB)

 Non-trainable params: 0 (0.00 B)

In [38]:
model.compile(optimizer='rmsprop',loss='categorical_crossentropy',metrics=['accuracy'])

In [39]:
final_x_d.shape,final_y.shape
print(type(final_x_d))
print(type(final_y))

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


In [51]:
len(final_x_d),len(final_y)

(61254, 61254)

In [52]:
# Use a small subset of the data
small_x = final_x_d[:10000]
small_y = final_y[:10000]

# Fit the model
model.fit(small_x, small_y, batch_size=32, epochs=5, validation_split=0.2)



Epoch 1/5
250/250 ━━━━━━━━━━━━━━━━━━━━ 47s 189ms/step - accuracy: 0.9955 - loss: 0.0135 - val_accuracy: 0.9940 - val_loss: 0.0177
Epoch 2/5
250/250 ━━━━━━━━━━━━━━━━━━━━ 46s 185ms/step - accuracy: 0.9959 - loss: 0.0123 - val_accuracy: 0.9941 - val_loss: 0.0175
Epoch 3/5
250/250 ━━━━━━━━━━━━━━━━━━━━ 46s 186ms/step - accuracy: 0.9962 - loss: 0.0114 - val_accuracy: 0.9942 - val_loss: 0.0171
Epoch 4/5
250/250 ━━━━━━━━━━━━━━━━━━━━ 47s 187ms/step - accuracy: 0.9963 - loss: 0.0109 - val_accuracy: 0.9943 - val_loss: 0.0169
Epoch 5/5
250/250 ━━━━━━━━━━━━━━━━━━━━ 188s 754ms/step - accuracy: 0.9965 - loss: 0.0102 - val_accuracy: 0.9944 - val_loss: 0.0169


In [54]:
import pickle

In [74]:
Mdl1 = pickle.dump(model,open('pos4.pkl','wb'))
Mdl1 =pickle.load(open('pos4.pkl','rb'))

In [70]:
Md3 = pickle.dump(tk_x,open('tk_x4.pkl','wb'))
Md3 =pickle.load(open('tk_x4.pkl','rb'))

In [58]:
Md2 = pickle.dump(tk_y,open('tk_y4.pkl','wb'))
Md2 =pickle.load(open('tk_y4.pkl','rb'))

In [94]:
te = ['India, a country located in South Asia, is the second-most populous country in the world. It is a federal republic with a parliamentary']


In [95]:
Md3.texts_to_sequences(te)

[[2086,
  8,
  292,
  1784,
  9,
  386,
  2081,
  11,
  2,
  272,
  94,
  13149,
  292,
  9,
  2,
  134,
  17,
  11,
  8,
  330,
  2794,
  18,
  8,
  10538]]

In [96]:
test = pad_sequences(Md3.texts_to_sequences(te),maxlen=271,padding="post")

In [97]:
model.predict(test).shape

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 119ms/step


(1, 271, 13)

In [67]:
import numpy as np

In [90]:
np.argmax(Mdl1.predict(test)[0],axis=1)[np.argmax(Mdl1.predict(test)[0],axis=1)!=0]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step


array([1, 2, 4, 6, 1, 1, 5, 6, 6, 6, 1, 1], dtype=int64)

In [98]:
pred = Md2.sequences_to_texts([list(np.argmax(Mdl1.predict(test)[0],axis=1)[np.argmax(Mdl1.predict(test)[0],axis=1)!=0])])


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step


In [99]:
pred

['noun det noun verb adp adj noun verb det adj adv adj noun adp det noun pron verb det adj noun adp det adj']

In [ ]:
import streamlit as st
import numpy as np
import pickle
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
st.image(r"C:\Users\ravin\Downloads\speech.webp")
Md2 = pickle.load(open('tk_y4.pkl', 'rb'))
Md3 = pickle.load(open('tk_x4.pkl', 'rb'))
Mdl1 = pickle.load(open('pos4.pkl', 'rb'))
test = st.text_input("Enter your input sequence")
if st.button("Predict"):
    test_tokens = test.split() 
    sequences = Md3.texts_to_sequences([test])  
    te = pad_sequences(sequences, maxlen=271, padding="post")
    pred = Mdl1.predict(te)
    pred_sequence = np.argmax(pred[0], axis=1)
    pred_sequence = pred_sequence[pred_sequence != 0]
    prediction_labels = Md2.sequences_to_texts([pred_sequence])[0].split()
    for token, label in zip(test_tokens, prediction_labels):
        st.title(f"{token} - {label}")


In [ ]:
import streamlit as st
import numpy as np
import pickle
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Load your models and tokenizer
Md2 = pickle.load(open('tk_y4.pkl', 'rb'))
Md3 = pickle.load(open('tk_x4.pkl', 'rb'))
Mdl1 = pickle.load(open('pos4.pkl', 'rb'))

st.image(r"C:\Users\ravin\Downloads\speech.webp")

test = st.text_input("Enter your input sequence")

if st.button("Predict"):
    if test:
        test_tokens = test.split()
        sequences = Md3.texts_to_sequences([test])
        te = pad_sequences(sequences, maxlen=271, padding="post")
        pred = Mdl1.predict(te)
        if pred.shape[0] > 0:
            pred_sequence = np.argmax(pred[0], axis=1)
            pred_sequence = pred_sequence[pred_sequence != 0]
            if len(pred_sequence) > 0:
                prediction_labels = Md2.sequences_to_texts([pred_sequence])[0].split()
                for token, label in zip(test_tokens, prediction_labels):
                    st.title(f"{token} - {label}")
            else:
                st.write("No valid predictions found.")
        else:
            st.write("Prediction array is empty.")
    else:
        st.write("Please enter an input sequence.")
